# Preprocessing English typing test data

In [1]:
import pandas as pd
import numpy as np
import csv

## Select participants
Select users who 
1. typed with mobile devices 
2. finished the whole test with 15 sentences 

In [3]:
# Load participants
participants = pd.read_csv('data/raw_data/typingtest_english_2020-06-03/raw_participants_en_2020-06-03.csv',quoting=2, quotechar='"', escapechar='\\',na_values=['N'], encoding='ISO-8859-1', low_memory=False)

participants = participants[['PARTICIPANT_ID', 'BROWSER', 'DEVICE', 'SCREEN_W','SCREEN_H', 'AGE','GENDER', 'HAS_TAKEN_TYPING_COURSE','WPM','ERROR_RATE', 'NATIVE_LANGUAGE','KEYBOARD_TYPE','USING_APP', 'USING_FEATURES','FINGERS','TIME_SPENT_TYPING', 'TYPE_ENGLISH']]

print(participants.head())
participants.shape[0]

   PARTICIPANT_ID BROWSER DEVICE  SCREEN_W  SCREEN_H   AGE  GENDER  \
0             1.0  Chrome    NaN       NaN       NaN  23.0  female   
1             2.0  Chrome    NaN       NaN       NaN   NaN     NaN   
2             3.0  Chrome    NaN       NaN       NaN   NaN     NaN   
3             4.0  Chrome    NaN       NaN       NaN   0.0    male   
4             5.0  Chrome    NaN       NaN       NaN   0.0    none   

   HAS_TAKEN_TYPING_COURSE        WPM  ERROR_RATE NATIVE_LANGUAGE  \
0                      0.0  40.532144    6.392694              ab   
1                      NaN        NaN         NaN             NaN   
2                      NaN        NaN         NaN             NaN   
3                      1.0  22.469162    0.796813              ab   
4                      1.0  18.685961    0.537634              ab   

  KEYBOARD_TYPE USING_APP                   USING_FEATURES       FINGERS  \
0        mobile     false  ["autocompletion","prediction"]         other   
1           

399833

In [8]:
# Select only mobile users
participants_mobile = participants.loc[participants['KEYBOARD_TYPE'] == 'mobile']

# Remove WPM != null
participants_mobile = participants_mobile[participants_mobile.WPM.apply(lambda x: x != 'N')]


# 0 < WPM < 200, error_rate < 25%
participants_mobile = participants_mobile.loc[participants_mobile['WPM'] > 0.]
participants_mobile = participants_mobile.loc[participants_mobile['WPM'] < 200.]
participants_mobile = participants_mobile.loc[participants_mobile['ERROR_RATE'] < 25.]

# PARTICIPANT_ID > 607 : Testers
participants_mobile = participants_mobile.loc[participants_mobile['PARTICIPANT_ID'] > 607]

# Between ages 10 and 70 (most of the data is between this age group)
participants_mobile = participants_mobile.loc[participants_mobile['AGE'] >= 10]
participants_mobile = participants_mobile.loc[participants_mobile['AGE'] <= 70]

participants = participants[pd.notnull(participants["BROWSER"])]

print(participants_mobile.head())
print(participants_mobile.head().dtypes)

     PARTICIPANT_ID                                            BROWSER  \
628           629.0  Mozilla/5.0 (Linux; Android 5.0.2; vivo Y51 Bu...   
672           673.0  Mozilla/5.0 (Linux; Android 6.0; HTC One X9 du...   
673           674.0  Mozilla/5.0 (iPhone; CPU iPhone OS 11_4_1 like...   
685           686.0  Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like M...   
687           688.0  Mozilla/5.0 (Linux; Android 5.1.1; A37f Build/...   

     DEVICE  SCREEN_W  SCREEN_H   AGE  GENDER  HAS_TAKEN_TYPING_COURSE  \
628  mobile     360.0     640.0  20.0    male                      0.0   
672  mobile     360.0     640.0  27.0  female                      0.0   
673  mobile     375.0     667.0  24.0  female                      0.0   
685  mobile     414.0     736.0  24.0    male                      0.0   
687  mobile     360.0     640.0  27.0  female                      1.0   

           WPM  ERROR_RATE NATIVE_LANGUAGE KEYBOARD_TYPE USING_APP  \
628  36.678613    0.934579              

In [9]:
print(participants_mobile.shape)

(62198, 17)


In [10]:
# Create new csv
participants_mobile.to_csv('data/processed2020/english/participants.csv', index=False)

# Clean test sections

Keep only selected participants.



In [11]:
# Load test_sections table
test_sections = pd.read_csv('data/raw_data/typingtest_english_2020-06-03/raw_test_sections_en_2020-06-03.csv', quoting=2, quotechar='"', escapechar='\\', encoding='ISO-8859-1', low_memory=False)
test_sections = test_sections[['TEST_SECTION_ID', 'SENTENCE_ID', 'PARTICIPANT_ID','USER_INPUT', 'WPM', 'ERROR_RATE', 'INPUT_TIME','EDIT_DISTANCE']]

print(test_sections.head())

   TEST_SECTION_ID  SENTENCE_ID  PARTICIPANT_ID  \
0              1.0        901.0             1.0   
1              2.0       1348.0             1.0   
2              3.0       1252.0             1.0   
3              4.0        978.0             1.0   
4              5.0        517.0             1.0   

                                          USER_INPUT                 WPM  \
0                   It's not looking too good is it?  45.510154147296305   
1  Once state owned, Telecom is now half owned by...   41.02665741750074   
2  These factors dictate creak limits to the conc...  30.539446785431185   
3                         I have forwarded to Kelly   57.361376673040155   
4                        Thank to you soon, Gerrard    59.88483685220729   

           ERROR_RATE INPUT_TIME EDIT_DISTANCE  
0                   0       8174             0  
1                   0      20182             0  
2                6.25      23969             4  
3  3.8461538461538463       5230        

In [12]:
# List of participants
users = participants_mobile['PARTICIPANT_ID'].tolist()

# Select only test sections of selected participants
test_sections_mobile = test_sections.loc[test_sections['PARTICIPANT_ID'].isin(users)]
test_sections_mobile = test_sections_mobile[test_sections_mobile.WPM.apply(lambda x: x != 'N')]
print(test_sections_mobile.shape)

(1217652, 8)


In [13]:
# Change data types
test_sections_mobile[['TEST_SECTION_ID', 'SENTENCE_ID', 'PARTICIPANT_ID']] = test_sections_mobile[['TEST_SECTION_ID', 'SENTENCE_ID', 'PARTICIPANT_ID']].apply(pd.to_numeric, downcast='integer')
test_sections_mobile[["WPM", "ERROR_RATE", 'INPUT_TIME','EDIT_DISTANCE']] = test_sections_mobile[["WPM", "ERROR_RATE", 'INPUT_TIME','EDIT_DISTANCE']].apply(pd.to_numeric)

In [14]:
#print(test_sections_mobile.head())
print(test_sections_mobile.head().dtypes)

TEST_SECTION_ID      int32
SENTENCE_ID          int16
PARTICIPANT_ID       int32
USER_INPUT          object
WPM                float64
ERROR_RATE         float64
INPUT_TIME           int64
EDIT_DISTANCE        int64
dtype: object


## Remove improper test sections and users

In [15]:
test_sections_mobile = test_sections_mobile.loc[test_sections_mobile['WPM'] > 0.]
test_sections_mobile = test_sections_mobile.loc[test_sections_mobile['WPM'] < 200.]
test_sections_mobile = test_sections_mobile.loc[test_sections_mobile['ERROR_RATE'] < 25.]
test_sections_mobile.shape

(1204770, 8)

In [16]:
# Select only users who have done at least 15 sentences.
selected_users = []
users_over15ts = []
for user in users:
    user_tests = test_sections_mobile.loc[test_sections_mobile['PARTICIPANT_ID'] == int(user)]
    num_tests = user_tests.shape[0]
    if num_tests >= 15: 
        selected_users.append(user)
        if num_tests > 15:
            users_over15ts.append(user)
    
print(len(selected_users))
print(len(users_over15ts))

test_sections_mobile = test_sections_mobile.loc[test_sections_mobile['PARTICIPANT_ID'].isin(selected_users)]
participants_mobile = participants_mobile.loc[participants_mobile['PARTICIPANT_ID'].isin(selected_users)]

57950
14368


In [17]:
test_sections_mobile.shape

(1147900, 8)

In [18]:
# Remove additional test sections (if participant has more than 15)
remove_ts_ids = []
for user in users_over15ts:
    user_tests = test_sections_mobile.loc[test_sections_mobile['PARTICIPANT_ID'] == int(user)]
    user_tests_list = user_tests['TEST_SECTION_ID'].tolist()
    user_tests_list.sort()
    for i in range(15,len(user_tests_list)):
        remove_ts_ids.append(user_tests_list[i])
        #print(i)
    
#remove ids from data frame
test_sections_mobile.drop(test_sections_mobile.loc[test_sections_mobile['TEST_SECTION_ID'].isin(remove_ts_ids)].index, inplace = True)
test_sections_mobile.shape

(869250, 8)

In [19]:
print("Mean:", test_sections_mobile['WPM'].mean())
print('Number of test sections', test_sections_mobile.shape)

Mean: 41.31580577593401
Number of test sections (869250, 8)


## Save CSV

In [20]:
# Create new csv for TEST_SECTIONS
test_sections_mobile.to_csv('data/processed2020/english/test_sections.csv', index=False)

In [21]:
# Create new csv for PARTICIPANTS
participants_mobile.to_csv('data/processed2020/english/participants.csv', index=False)

# Clean logs



In [22]:
# Selected test sections
test_section_list = test_sections_mobile['TEST_SECTION_ID'].tolist()

In [24]:
# Read in chunks

first_chunk = True
#col_name = ['LOG_DATA_ID','TEST_SECTION_ID','TYPE','KEY','CODE','DATA','INPUT','PRESSED','TIMESTAMP','DEVICE_ORIENTATION','SCREEN_ORIENTATION']
for chunk in pd.read_csv('data/raw_data/typingtest_english_2020-06-03/raw_log_data_en_2020-06-03.csv', chunksize=1000000, quoting=2, quotechar='"', escapechar='\\', encoding='ISO-8859-1', low_memory=False):
    #process(chunk)
    
    
    #print("mem1 original", chunk.memory_usage(index=True).sum())
    # Transform data types
    chunk[['LOG_DATA_ID', 'TEST_SECTION_ID']] = chunk[['LOG_DATA_ID', 'TEST_SECTION_ID']].apply(pd.to_numeric, downcast='integer')
    chunk[['TIMESTAMP']] = chunk[['TIMESTAMP']].apply(pd.to_numeric)
    chunk[["TYPE", "KEY", "EVENT_CODE"]] = chunk[["TYPE", "KEY", "CODE"]].astype('category')

    #print(chunk.head().dtypes)
    #print("mem2 data types", chunk.memory_usage(index=True).sum())
        
    # Select only wanted test sections
    chunk = chunk.loc[chunk['TEST_SECTION_ID'].isin(test_section_list)]

    #print("mem3 select test sections", chunk.memory_usage(index=True).sum())
    
    # Write as CSV file.
    #chunk.to_csv('data/processed2019/log_chunks/log_data_mobile.csv', mode='a')
    
    # Keep only keyups
    chunk_filtered = chunk.loc[chunk['TYPE'] == 'keyup']
    
    #print("mem4 only keyup", chunk_filtered.memory_usage(index=True).sum())
    
    # Remove unnecessary columns.
    chunk_filtered = chunk_filtered[['TEST_SECTION_ID', 'LOG_DATA_ID', 'TIMESTAMP', 'KEY', 'EVENT_CODE','INPUT']]
    
    #print("mem5 remove cols", chunk_filtered.memory_usage(index=True).sum())

    # Order by TEST_SECTION_ID and TIMESTAMP (Order after creating new tables)
    chunk_filtered.sort_values(by=['TEST_SECTION_ID','TIMESTAMP'])
    
    # Write as CSV file.
    if first_chunk:
        chunk_filtered.to_csv('data/processed2020/english/log_data.csv', mode='w', index=False)
        first_chunk = False
    else:
        chunk_filtered.to_csv('data/processed2020/english/log_data.csv', mode='a', index=False, header=None)
        

# concat the list into dataframe 
#df_concat = pd.concat(chunk_list)

